## Imports and Jupyter Config

In [215]:
#general imports
import pandas as pd
import numpy as np

#nltk imports
import nltk as nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

#wordcloud imports
import os
import matplotlib.pyplot as plt


#set options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',None)

ImportError: cannot import name 'WordCloud' from 'word_cloud' (unknown location)

## Functions to be Reused

In [204]:
def GetVerseReferences(verseReferenceListing):
    '''Given a book, chapter, verse listing, return three lists that contain those elements broken out in that order.'''
    
    verseReferenceListing = [ele[::-1] for ele in verseReferenceListing]
    
    verses = []
    chapters = []
    books = []

    for ele in verseReferenceListing:
        temp_list_1 = ele.split(':')
        verses.append(temp_list_1[0][::-1])
        temp_list_2 = temp_list_1[1].split(' ',1)
        chapters.append(temp_list_2[0][::-1])
        books.append(temp_list_2[1][::-1])

    return books, chapters, verses

def GetMaxVerseLength(verseListing):
    '''Given a list of verse texts, return the maximum verse length in terms of word count along with a dictionary of verse references and their lengths'''
    lenDict = {}
    counter = 0
    for ele in verseListing:
        temp_list = ele.split(' ') 
        lenDict[counter] = len(temp_list)
        counter = counter + 1
    maxLen = max(lenDict.values())
    return maxLen, lenDict

def IndicateWordPresence(bible_dataframe_verse_series, word_list):
    '''given a dataframe column containing verse texts and a list of pertinent words, return a column as a list that indicates for each verse whether one of those words is present --- boolean column'''
    word_indicator = [] #initialize list indicating whether words are present in verses
    for ele in bible_dataframe_verse_series:
        if any(item in word_list for item in word_tokenize(ele)):
            word_indicator.append(True)
        else: word_indicator.append(False)
    return word_indicator

def LoadBible(bible_excel_file):
    '''given an excel file containing the bible, return a dataframe for analysis'''
    
    #generally format intake
    bible_columns = 'BookChapterVerse','VerseText'
    bible = pd.read_excel(bible_excel_file).drop([0]).reset_index(drop=True)
    bible.columns = bible_columns
    bookChapterVerse = bible.BookChapterVerse
    book, chapter, verse = GetVerseReferences(bookChapterVerse)
    bible['Book'] = book
    bible['Chapter'] = chapter
    bible['Verse'] = verse

    #trim verses to key words
    stop_words = list(stopwords.words('english'))
    stop_words.extend([',','.',';',':']) #add extra symbols to be removed
    verse_text_no_stopwords = []
    for verse in bible.VerseText:
        verse_words = []
        for word in word_tokenize(verse):
            if word.casefold() not in stop_words:
                verse_words.append(word.lower())
        verse_text_no_stopwords.append(verse_words)
    bible['VerseText_noStopWords'] = verse_text_no_stopwords

    stemmer = PorterStemmer()
    verses_stemmed = []
    for verse in verse_text_no_stopwords:
        verse_stemmed = []
        for word in verse:
            verse_stemmed.append(stemmer.stem(word))
        verses_stemmed.append(verse_stemmed)
    bible['VerseText_noStopWords_stemmed'] = verses_stemmed
    return bible

## User Variables to be Referenced

In [156]:
tree_words = ['tree','forest','shrug','bark','eucalyptus','oak','birch','pine','root','acacia','beech','trunk','leaf','grove','seed']

## Iterative Exploratory Analysis

In [205]:
akjv = LoadBible('akjv.xlsx')
# asv = LoadBible('asv.xlsx')
# bsb = LoadBible('bsb.xlsx')
# cpdv = LoadBible('cpdv.xlsx')
# kjv = LoadBible('kjv.xlsx')
# ylt = LoadBible('ylt.xlsx')

In [206]:
akjv.head()

,BookChapterVerse,VerseText,Book,Chapter,Verse,VerseText_noStopWords,VerseText_noStopWords_stemmed
0,Genesis 1:1,In the beginning God created the heaven and the earth.,Genesis,1,1,"[beginning, god, created, heaven, earth]","[begin, god, creat, heaven, earth]"
1,Genesis 1:2,"And the earth was without form, and void; and darkness was on the face of the deep. And the Spirit of God moved on the face of the waters.",Genesis,1,2,"[earth, without, form, void, darkness, face, deep, spirit, god, moved, face, waters]","[earth, without, form, void, dark, face, deep, spirit, god, move, face, water]"
2,Genesis 1:3,"And God said, Let there be light: and there was light.",Genesis,1,3,"[god, said, let, light, light]","[god, said, let, light, light]"
3,Genesis 1:4,"And God saw the light, that it was good: and God divided the light from the darkness.",Genesis,1,4,"[god, saw, light, good, god, divided, light, darkness]","[god, saw, light, good, god, divid, light, dark]"
4,Genesis 1:5,"And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.",Genesis,1,5,"[god, called, light, day, darkness, called, night, evening, morning, first, day]","[god, call, light, day, dark, call, night, even, morn, first, day]"
